In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

In [3]:
import sys
!{sys.executable} -m pip install sklearn

     |████████████████████████████████| 7.3 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 7.8 MB/s eta 0:00:01
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn ... done
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.16/libexec/bin/python3.9 -m pip install --upgrade pip' command.


In [4]:
from sklearn import linear_model

In [5]:
def clean_data(df):
    df.columns=[e.lower().replace(' ', '_') for e in df.columns]
    print(df.columns)
    df=df.dropna()
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df

In [8]:
data = pd.read_csv("alldata.csv")
data = clean_data(data)
# data = data.drop("description", axis=1)
data["reviews"] = data.reviews.astype(int)

Index(['position', 'company', 'description', 'reviews', 'location'], dtype='object')


In [9]:
data.describe(include='all')
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5326 entries, 0 to 5325
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   position     5326 non-null   object
 1   company      5326 non-null   object
 2   description  5326 non-null   object
 3   reviews      5326 non-null   int64 
 4   location     5326 non-null   object
dtypes: int64(1), object(4)
memory usage: 56.2 MB


In [10]:
# check for null
data.isna().all(axis=0)
data.isna().all(axis=1)

0       False
1       False
2       False
3       False
4       False
        ...  
5321    False
5322    False
5323    False
5324    False
5325    False
Length: 5326, dtype: bool

In [12]:
# data.value_counts()

# split categorical Features and Numerical Features.
numerical = data._get_numeric_data()
categorical = data[set(data.columns) - set(numerical.columns)]

numerical.describe(include="all")
numerical.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5326 entries, 0 to 5325
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   reviews  5326 non-null   int64
dtypes: int64(1)
memory usage: 41.7 KB


In [13]:
# find the unique values in each column
[data[c].value_counts() for c in data.columns.drop("description")]

[Data Scientist                                            204
 Senior Data Scientist                                      53
 Research Analyst                                           44
 Data Engineer                                              39
 Machine Learning Engineer                                  26
                                                          ... 
 Development Associate, Donor Systems - Data Management      1
 Senior Software Engineer, Cloud Engineering                 1
 Lead Data Scientist - Ad Tech / Marketing                   1
 Manager, Data and Analysis                                  1
 Senior Software Engineer - Platform Team                    1
 Name: position, Length: 4221, dtype: int64,
 Amazon.com            357
 Ball Aerospace        187
 Microsoft             137
 Google                134
 NYU Langone Health     76
                      ... 
 SKA                     1
 Quidel Corporation      1
 XL Catlin               1
 Amyx Inc          

In [14]:
### Who gets hired? What kind of talent do employers want when they are hiring a data scientist?
# from scipy import stats

print(data["position"].value_counts()) 
print("the position with the highest demand is the data scientist position")



Data Scientist                                            204
Senior Data Scientist                                      53
Research Analyst                                           44
Data Engineer                                              39
Machine Learning Engineer                                  26
                                                         ... 
Development Associate, Donor Systems - Data Management      1
Senior Software Engineer, Cloud Engineering                 1
Lead Data Scientist - Ad Tech / Marketing                   1
Manager, Data and Analysis                                  1
Senior Software Engineer - Platform Team                    1
Name: position, Length: 4221, dtype: int64
the position with the highest demand is the data scientist position


In [18]:
### Which location has the most opportunities?
print(data["location"].value_counts().sort_values(ascending = False))
print("%d positions were offered in Seattle" %
      data["location"].value_counts().sort_values(ascending = False)[0])

Seattle, WA                474
Cambridge, MA              368
New York, NY               338
Boston, MA                 305
San Francisco, CA          287
                          ... 
San Francisco, CA 94122      1
Atlanta, GA 30329            1
New York, NY 10168           1
Atlanta, GA 30339            1
Orangeburg, NY 10962         1
Name: location, Length: 328, dtype: int64
474 positions were offered in Seattle


In [19]:
data["location"].value_counts().sum()
data["location"].value_counts().sort_values(ascending = False)

Seattle, WA                474
Cambridge, MA              368
New York, NY               338
Boston, MA                 305
San Francisco, CA          287
                          ... 
San Francisco, CA 94122      1
Atlanta, GA 30329            1
New York, NY 10168           1
Atlanta, GA 30339            1
Orangeburg, NY 10962         1
Name: location, Length: 328, dtype: int64

In [20]:
### What skills, tools, degrees or majors do employers want the most for data scientists?
skills = ["Python",
          "SQL",
          "Excel",
          "R",
          "Scala",
          "machine learning",
          "modeling",
          "optimization",
          "data visualization",
          "artificial intelligence",
          "aws"]


degrees = ["Computer Science", "Statistics", "Mathematics", "Economics", "Data Science"]

keywords = skills + degrees


In [21]:
# data_scientist_positions
dsp = data[data["position"].str.match('.*(?i)data.*' )]

In [22]:
# equivalent labels for "Data Scientist"
names = dsp["position"].value_counts()

In [23]:
# r = '(.*(?i)tool.*)|(.*(?i)degree.*)|(.*(?i)bachelor.*)|(.*(?i)phd.*)'

# data[data["position"].str.match('.*(?i)Python.*' )]
# data[data["position"].str.match(r)]

def mkreg(arg):
    return '.*(?i)' + arg + ".*"


In [25]:
# kw_stats represents the frequency of each keyword in the description columns of "data scientist" positions
kw_stats = pd.DataFrame()
for kw in keywords:
    matches = len(dsp[dsp["description"].str.match(mkreg(kw))])
    kw_stats[kw] = [matches]


In [27]:
print(kw_stats.describe())

       Python   SQL  Excel       R  Scala  machine learning  modeling  \
count     1.0   1.0    1.0     1.0    1.0               1.0       1.0   
mean     41.0  37.0  114.0  1390.0   20.0             106.0      52.0   
std       NaN   NaN    NaN     NaN    NaN               NaN       NaN   
min      41.0  37.0  114.0  1390.0   20.0             106.0      52.0   
25%      41.0  37.0  114.0  1390.0   20.0             106.0      52.0   
50%      41.0  37.0  114.0  1390.0   20.0             106.0      52.0   
75%      41.0  37.0  114.0  1390.0   20.0             106.0      52.0   
max      41.0  37.0  114.0  1390.0   20.0             106.0      52.0   

       optimization  data visualization  artificial intelligence   aws  \
count           1.0                 1.0                      1.0   1.0   
mean           78.0                14.0                     34.0  16.0   
std             NaN                 NaN                      NaN   NaN   
min            78.0                14.0       

In [28]:
data[data["position"].str.match(mkreg("Python"))]

,position,company,description,reviews,location
721,Senior Data Scientist / Python Developer,WinterWyman,One of the region's premier investment managem...,30,"Boston, MA 02111"
980,Data Engineer with Python Experience,Workbridge Associates,A successful consulting firm located in Cambri...,40,"Boston, MA"
1133,Lead Data Scientist - Python SME,84.51°,We are a community of analysts that act as the...,2,"Chicago, IL"
1151,"Mid-Level Data Scientist (Python, Pandas, NumP...",Workbridge Associates,A thriving company located in Downtown Chicago...,40,"Chicago, IL"
1280,"Mid to Senior Level Data Engineer (Python, Spa...",Workbridge Associates,A quicken loans company located in the River N...,40,"Chicago, IL"
2205,Python Software Developer - AMAZON ALEXA Machi...,Amazon.com,Job Description\nPython Software Development E...,25902,"Cambridge, MA"
2541,Test Engineer (Machine Learning) with Python o...,"GlobalLogic, Inc.",Job Description\n\nMinimum qualifications:\n\n...,353,"Mountain View, CA"
2782,Python Developer,Nielsen,Everyone knows about the Nielsen Ratings - but...,1918,"New York, NY 10003"
3096,Data Reporting Developer (Python) - Accounting,Hudson River Trading,Hudson River Trading's Accounting team is look...,2,"New York, NY 10005"
3255,Data Engineer Python & Spark,Mount Sinai Health System,Strength Through DiversityGround breaking scie...,1029,"New York, NY 10019"
